In [23]:
import torch
import pandas as pd
import numpy as np

In [24]:
df=pd.read_csv('Bank_Personal_Loan_Modelling-Copy1.csv')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [26]:
df.head()


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [27]:
df.drop("ZIP Code",axis=1,inplace=True)
df.drop("ID",axis=1,inplace=True)

In [28]:
df['target']=df['Personal Loan']


In [29]:
df.drop('Personal Loan',axis=1,inplace=True)

In [30]:
df["Experience"] = abs(df["Experience"])

In [31]:
df

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,target
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,3,0,0,0,1,0,0
4996,30,4,15,4,0.4,1,85,0,0,1,0,0
4997,63,39,24,2,0.3,3,0,0,0,0,0,0
4998,65,40,49,3,0.5,2,0,0,0,1,0,0


In [32]:
x = df.iloc[:,:-1].values
y= df.iloc[:,-1].values

In [33]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.3,random_state =0)

In [34]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [35]:
import torch
from torch.utils.data import DataLoader, TensorDataset
BATCH_SIZE = 2
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)


In [36]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

In [37]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

In [38]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,16)
        self.layer2 = torch.nn.Linear(16,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x


In [39]:
model = Model()

In [40]:
import numpy as np
from typing import Tuple
import torch


class ParticleSwarmOptimizer:
    def __init__(self, model: torch.nn.Module, w: float, c1: float, c2: float, num_of_particles: int, decay: float,
                 inputs: torch.Tensor, labels: torch.Tensor) :
        
        self.model = model
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.num_of_particles = num_of_particles
        self.inputs = inputs
        self.labels = labels
        self.positions = self.initialize_position()
        self.velocities = self.initialize_velocity()
        self.pbest = self.positions
        self.gbest = np.inf
        self.decay = decay

    def initialize_position(self):
        
        num_of_weights = sum(p.numel() for p in self.model.parameters())
        return np.random.uniform(-1, 1, size=(self.num_of_particles, num_of_weights))

    def initialize_velocity(self):
        
        num_of_weights = sum(p.numel() for p in self.model.parameters())
        return np.random.uniform(-1, 1, size=(self.num_of_particles, num_of_weights))

    def train(self, num_of_iterations: int):
       
        for i in range(num_of_iterations):
            fitness_values = self.evaluate_fitness()
            self.update_personal_best(fitness_values)


In [41]:
def find_gbest(self):
        for position in self.positions:
            if self.fitness(position) < self.fitness(self.gbest):
                self.gbest =position

In [42]:
def new_velocity(self):
    
    num_params = sum(p.numel() for p in self.model.parameters())
    r1 = np.random.rand(self.num_of_particles, num_params)
    r2 = np.random.rand(self.num_of_particles, num_params)
    self.velocity = (self.w * self.velocity) + (self.c1 * r1 * (self.pbest - self.positions)) + (self.c2 * r2 * (self.gbest - self.positions))


In [43]:
def new_position():
    
    self.positions += self.velocity
    
def fitness(self, weights) -> float:
    
    self.model.load_state_dict(weights)
    outputs = self.model(self.inputs.float())
    loss = torch.nn.functional.binary_cross_entropy(outputs, self.labels.float())
    return -1.0 * loss.item()


In [44]:
from sklearn.metrics import classification_report
y_pred = model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.47      0.90      0.62       719
           1       0.41      0.07      0.12       781

    accuracy                           0.46      1500
   macro avg       0.44      0.48      0.37      1500
weighted avg       0.44      0.46      0.36      1500

